# Datová analýza exoplanetárního datasetu</h1>
## Semestrální práce z předmětu 4ST101
### *Autor: Jakub Vítovec*

## Úvod
Ve své semestrální práci se budu zabývat daty získanými z veřejné databáze [Keggle](https://www.kaggle.com/). Konkrétní dataset je dostupný na URL [https://www.kaggle.com/datasets/sathyanarayanrao89/nasa-exoplanetary-system](https://www.kaggle.com/datasets/sathyanarayanrao89/nasa-exoplanetary-system) a byl vytvořen uživatelem Sathyanarayan Rao z dat publikovaných vesmírnou agenturou NASA. Celkem se v datasetu nachází 32552 různých dosud objevených exoplanet a základní data o nich.
Exoplaneta je planeta, která obíhá okolo jiné hvězdy, než okolo Slunce. První exoplaneta byla objevena v roce 1998 a počet jejich počet stále roste. Ve své semestrální práci bych se rád zaměřil na trend růstu či poklesu počtu objevených nových exoplanet a zapojení různých vědeckých zařízení při jejich objevování.

### Představení dat a jejich čištění
V tomto oddíle si představíme zpracovávaná data a vyčistíme je od nežádoucích hodnot.

In [115]:
import pandas as pd;
import matplotlib.pyplot as plt;
import numpy as np;
from scipy.stats import skew;

dataset = pd.read_csv("./exoplanets_dataset.csv");
dataset.info()

Jak vidíme dataset obsahuje značné množství nepojmenovaných sloupců - Unnamed. Ty z datasetu odstraníme jako první.

In [116]:
for column_name in dataset.columns:
    if ("Unnamed" in column_name):
        del dataset[column_name];

Jako další by bylo vhodné zkontrolovat počty chybějících hodnot v jednotlivých sloupcích.

In [117]:
print(dataset.isnull().sum());

Nejvíce chybějících hodnot se nachází ve sloupcích Planet Mass or Mass*sin(i), Eccentricity a Planet Radius, které obsahují hmotnost planety, ekcentricitu její oběžné dráhy a jejich poloměr. Vzhledem k tomu, že značná část těchto dat nám chybí, tak nebudeme tyto hodnoty využívat a sloupce odstraníme.

In [118]:
columns_to_delte = [" Planet Radius [Earth Radius]", "Planet Mass or Mass*sin(i) [Earth Mass]",  " Eccentricity"]

for column_name in dataset.columns:
    if (column_name in columns_to_delte):
        del dataset[column_name];

Jako další odstraníme z datasetu duplikované hodnoty.

In [119]:
print("Deleted duplicate values: " + str(dataset.duplicated().sum()));
dataset.drop_duplicates();

Po odstranění sloupců se značným počtem chybějících hodnot nám v datasetu zůstaly následující proměnné:
- **Planet Name** - Obsahuje jméno dané exoplanety
- **Host Name** - Jméno hostující hvězdy
- **Number of Stars** - Počer hvězd v planetárním systému dané exoplanety (u Země 1)
- **Number of Planets** - Počet planet v planetárním systému dané exoplanety (u Země 8)
- **Discovery Method** - Způsob objevení planety
- **Discovery Year** - Rok objevení planety
- **Discovary Facility** - Zařízení, které objevilo danou planetu
- **Orbital Period** - Orbitální perioda, doba v letech, za kterou planeta oběhne svou hostující hvězdu (Host Name)

## Teoretická část
V této části se zaměřím na statistické veličiny, které budou použity k analýze našeho datového souboru.
Jako základní veličinu míry polohy použijeme aritmetický průměr. Náš soubor obsahuje jak spojité, tak diskrétní náhodné veličiny. Ačkoliv jsou spojité fyzikální veličiny při reálných měřeních často interpretovány jako diskrétní, vzhledem k tomu, že rozptyl u planetárních měření je zpravidla podstatně větší, než nejmenší rozlišitelná hodnota, tak bude vhodné tuto veličinu považovat za spojitou.

- Aritmetický průměr spojité náhodné veličiny *E(x)* spočítáme jako:
$$E(x)=\int_{D(f)}xf(x)dx$$

- Aritmetický průměr diskrétní náhodné veličiny *E(x)* spočítáme jako:
$$E(x)=1/N\displaystyle\sum\limits_{i=0}^N x_{i}$$

Vzhledem k tomu, že jsme se rozhodli zkoumat tredy vývoje objevování nových planet, tak budeme potřebovat statistickou veličinu, kterou tento vývoj kvantifikujeme. K tomu je vhodné použít řetězové a bazické indexy. Pomocí nich můžeme snadno měřit růst či pokles dané veličiny mezi po sobě jdoucími obdobími nebo vůči nějakému základnímu období.

- Řetězový index růstu veličiny *x* spočítáme jako:
$$I_{t-1}^{t}=\frac{x_{t}}{x_{t-1}}$$

- Bazický index růstu veličiny *x* vůči pevnému základu spočítáme jako:
$$I_{t_{0}}^{t}=\frac{x_{t}}{x_{t_{0}}}$$

Když jsme zavedli aritmetický průměr pro měření polohy a indexy pro zkoumání trendů, bude vhodné zavést také možnost, jak spočítat průměr nějakých indexů za dané období, tím získáme střední hodnotu růstu či poklesu dané proměnné. K průměrování indexů se používá geometrický průměr, ten je definován, jako n-tá odmocnina z produktu daných čísel.

- Geometrický průměr spočítáme jako:
$$E_{g}(x)=\left(\prod _{{i=1}}^{{n}}x_{i}\right)^{{{\frac {1}{n}}}}$$

K zobrazení rozdělení četnosti použijeme histogram. Histogram je druh sloupcového grafu, který znázorňuje distribuci dat tak, že výška sloupce určuje četnost dané veličiny. U histogramu je kritické určení šířky intervalů (počtu tříd), do kterých dané četnosti rozdělíme. Pro Určení počtu tříd použijeme doporučený vztah
$$k=\lceil2.46(N-1)^{0,4}\rceil$$

Díky histogramu budeme schopni vizualizovat množství planet objevených jednotlivými zařízeními i vývoj počtu nově oběvených planet.

K analýze distribuce nově objevených planet použijeme také šikmost. Pomocí šikmosti prozkoumáme symetrii tohoto rozdělení.
- Šikmost rozdělení spočítáme jako Fisher-Personův koeficient šikmosti tohoto rozdělení
$$g_{1}=\frac{m_{3}}{m_{2}^{3/2}}$$
, kde
$$m_{i}=\frac{1}{N}\displaystyle\sum\limits_{n=1}^N (x[n]-\bar{x})^i$$
je vážený vzorek i-tého centrálního momentu. Pokud váha vyjde 0, použijeme Fisher-Personův stadnardizovaný koeficient momentu a hodnoty upravíme:
$$G_{1}=\frac{\sqrt{N(N-1)}}{N-2}\frac{m_{3}}{m_{2}^{3/2}}$$
Při použití tohoto vzorce nám hodnoty větší než nula značí vyšší váhu na pravé straně distribuované hodnoty.


## Praktická část
V rámci praktické části se zaměříme na samotné zpracování dat, pomocí kterého budeme zkoumat trendy v počtech nových objevených exoplanet. Pro zkoumání četností nově objevených planet v daných letech si musíme nejprve sestavit nový DataFrame obsahující počty planet objevené v daných letech. Dále sestavíme řetězový index tohoto sloupce a určíme jeho směrodatnou odchylku, aritmetický průměr a variační koeficient, abychom zjistili, jak moc se od sebe hodnoty vzájemě liší.

In [124]:
planet_discovery_frequencies = dataset.groupby("Discovery Year")["Planet Name"].count().reset_index(name="Frequency");
planet_discovery_frequencies["F/F-1"] = planet_discovery_frequencies["Frequency"]/planet_discovery_frequencies["Frequency"].shift(1);
print("Variační koeficient počtu nově objevených exoplanet ročně: " + str(planet_discovery_frequencies["Frequency"].std()/planet_discovery_frequencies["Frequency"].mean()))
planet_discovery_frequencies["Frequency"].describe()

Ze získaných hodnot je patrné, že variační koeficient je velice vysoký (přibližně 256%), tím pádem pro nás aritmetický průměr není vypovídající statistickou veličinou. Zároveň můžeme vypozorovat, že se maximum danového souboru nachází vysoko nad hodnotou třetího kvartilu, to nám indikuje, že exisuje několik málo roků, během kterých bylo objeveno neobvykle mnoho exoplanet. Abychom mohli vizuálně posoudit počty objevených planet v jednotlivých letech, vytvoříme histogram. Vzhledem k obrovskému variačnímu rozpětí souboru a vysokému variačnímu koeficientu bude na osu *y* znázorňující počet objevených planet vhodné použít logaritmické měřítko, díky němu bude mezi vzdálenými hodnotami graficky menší vzdálenost a celý graf bude přehlednější

In [121]:
plt.bar(planet_discovery_frequencies['Discovery Year'], planet_discovery_frequencies['Frequency'])
plt.xlabel('Rok objevení')
plt.ylabel('Počet nově objevených exoplanet (log měřítko)')
plt.title('Graf 1: Počet ročně objevených nových exoplanet')
plt.yscale('log');
y_vals = np.logspace(np.log10(1), np.log10(max(planet_discovery_frequencies['Frequency'])), num=5, endpoint=True, base=10)
for y_val in y_vals:
    plt.axhline(y=y_val, color='gray', linestyle='-', alpha=0.5, zorder=0)

plt.show()


Z Grafu 1. lze vyčíst, že data vykazují poměrně přirozený, vizuálně až logaritmický růst, s výjimkou let 2014 a 2016, během kterých bylo objevenou 8822 a 12718 nových exoplanet. Z řetězových indexů můžeme snadno vyčíst meziroční růst 950% a 1600%. Počty objevené jenom v těchto dvou letech tvoří podstatnou část všech objevených exoplanet, proto nemůžeme tyto hodnoty ignorovat a pokusíme se zodpovědět, co stojí za těmito počty. Vytvoříme si dva nové DataFramy obsahující planety objevené v letech 2014 & 2016 a všechny ostatní, seskupíme je podle vědeckého zařízení, které je objevilo a nalezneme 5 nejvyšších hodnot.

In [122]:
critical_years_planets = dataset[dataset["Discovery Year"].astype(str).str.contains("2014|2016")]
non_critical_years_planets = dataset[~dataset["Discovery Year"].astype(str).str.contains("2014|2016") ]
print(non_critical_years_planets.groupby("Discovery Facility")["Planet Name"].count().reset_index(name="Planets Discovered").sort_values("Planets Discovered").tail());
print("-" * 45);
print(critical_years_planets.groupby("Discovery Facility")["Planet Name"].count().reset_index(name="Planets Discovered").sort_values("Planets Discovered").tail());

Z výsledných dat je patrné, že v kritických letech 2014 a 2016 byla práce ostatních observatoří utlumena a dominantním objevitelem byla sonda Kepler. Tato čísla se dají snadno vysvětlit faktem, že sonda Kepler se po svém vypuštění stala v tomto roce plně funkční a ve své hlavní činnosti pokračovala až do roku 2017 a planety objevené po tomto roce jsou už převážně odhalené analýzou jejích starších dat. Vliv této mise můžeme zároveň ověřit výpočtem šikmosti distribuce nově objevených planet.

In [123]:
print("Šikmost distribuce nově objevených planet: " + str(skew(planet_discovery_frequencies['Frequency'])));

Hodnota této veličiny dosahuje přes 3.55, to indikuje silné zkreslení grafu směrem vpravo. Převažují tedy hodnoty menší než průměr a distribuce je silně asymetrická. To je způsobené rolí mise Kepler.

In [126]:
plt.scatter(dataset['Discovery Year'], dataset['Orbital Period'])
plt.show()